In [ ]:
import numpy as np
from itertools import product

transformations = [
    lambda x: x,                         # Identity
    lambda x: np.rot90(x, 1),            # Rotate 90°
    lambda x: np.rot90(x, 2),            # Rotate 180°
    lambda x: np.rot90(x, 3),            # Rotate 270°
    lambda x: np.fliplr(x),              # Horizontal reflection
    lambda x: np.flipud(x),              # Vertical reflection
    lambda x: np.transpose(x),           # Diagonal reflection (TL-BR)
    # lambda x: np.fliplr(np.transpose(x)) # Diagonal reflection (TR-BL)
]

inverse_transformations = [
    lambda x: x,                         # Identity
    lambda x: np.rot90(x, 3),            # Rotate 90° inverse (rotate 270°)
    lambda x: np.rot90(x, 2),            # Rotate 180° inverse
    lambda x: np.rot90(x, 1),            # Rotate 270° inverse (rotate 90°)
    lambda x: np.fliplr(x),              # Horizontal reflection inverse
    lambda x: np.flipud(x),              # Vertical reflection inverse
    lambda x: np.transpose(x),           # Diagonal reflection (TL-BR) inverse
    # lambda x: np.transpose(np.fliplr(x))# Diagonal reflection (TR-BL) inverse
]

original_actions = np.array(range(9)).reshape(3, 3)
for i, transform in enumerate(transformations):
    assert inverse_transformations[i](transform(original_actions)).flatten().tolist() == original_actions.flatten().tolist()

transformed_actions = [transform(original_actions).flatten().tolist() for transform in transformations]
for i in range(len(transformed_actions)):
    for j in range(i + 1, len(transformed_actions)):
        assert transformed_actions[i] != transformed_actions[j]

def generate_all_valid_boards():
    symbols = [' ', 'X', 'O']
    all_boards = list(product(symbols, repeat=9))  # Generate all 3^9 combinations
    print(f"Total number of boards: {len(all_boards)}")
    all_valid_boards = []

    for board in all_boards:
        x_count = board.count('X')
        o_count = board.count('O')
        
        # Valid boards must satisfy these conditions:
        if x_count == o_count or x_count == o_count + 1:
            all_valid_boards.append(board)

    print(f"Number of valid boards: {len(all_valid_boards)}")    
    return all_valid_boards

def board_to_matrix(board):
    return np.array(board).reshape(3, 3)

def matrix_to_board(matrix):
    return matrix.flatten().tolist()

def generate_symmetries(board):
    matrix = board_to_matrix(board)
    symmetries = [transform(matrix) for transform in transformations]
    return [matrix_to_board(sym) for sym in symmetries]

def get_canonical_representation(board):
    symmetries = generate_symmetries(board)
    min_symmetry = min(symmetries)
    return tuple(min_symmetry), symmetries.index(min_symmetry)

# Generate all empty positions on the board
def get_empty_positions(board):
    return [i for i, cell in enumerate(board) if cell == ' ']

# Generate all valid Tic-Tac-Toe boards
all_valid_boards = generate_all_valid_boards()
state_action_pairs = 0
for valid_board in all_valid_boards:
    empty_positions = get_empty_positions(valid_board)
    state_action_pairs += len(empty_positions)

print(f"Total number of valid state-action pairs: {state_action_pairs}")

# Generate all canonical Tic-Tac-Toe boards
all_canonical_boards = set()
get_canonical_board = {}
get_transform = {}
get_inverse_transform = {}
get_canonical_actions = {}
get_inverse_canonical_actions = {}
for valid_board in all_valid_boards:
    canonical_board, transform_idx = get_canonical_representation(valid_board)
    all_canonical_boards.add(canonical_board)
    get_canonical_board[valid_board] = canonical_board
    get_transform[valid_board] = transformations[transform_idx]
    get_inverse_transform[valid_board] = inverse_transformations[transform_idx]
    get_inverse_canonical_actions[valid_board] = get_transform[valid_board](original_actions).flatten().tolist()
    get_canonical_actions[valid_board] = get_inverse_transform[valid_board](original_actions).flatten().tolist()

all_canonical_boards = list(all_canonical_boards)
print(f"Number of canonical boards: {len(all_canonical_boards)}")

all_canonical_actions = {}
canonical_state_action_pairs = 0
for canonical_board in all_canonical_boards:
    empty_positions = get_empty_positions(canonical_board)
    all_canonical_actions[canonical_board] = empty_positions
    canonical_state_action_pairs += len(empty_positions)

print(f"Number of canonical state-action pairs: {canonical_state_action_pairs}")

def get_canonical_action(board, action):
    return get_canonical_actions[board][action]

def get_inverse_canonical_action(board, canonical_action):
    return get_inverse_canonical_actions[board][canonical_action]

valid_board = all_valid_boards[5]
actions = get_empty_positions(valid_board)
canonical_actions = [get_canonical_action(valid_board, action) for action in actions]
print(f"Board:             {valid_board}")
print(f"Canonical board:   {get_canonical_board[valid_board]}")
print(f"Actions:           {actions}")
print(f"Canonical actions: {sorted(canonical_actions)}")
print(f"Canonical actions: {sorted(all_canonical_actions[get_canonical_board[valid_board]])}")

tot = 0
for i, valid_board in enumerate(all_valid_boards):
    actions = get_empty_positions(valid_board)
    canonical_board = get_canonical_board[valid_board]
    canonical_actions1 = sorted(all_canonical_actions[canonical_board])
    canonical_actions2 = sorted([get_canonical_action(valid_board, action) for action in actions])
    if not canonical_actions2 == canonical_actions1:
        tot += 1

print(f"Number of wrong canonical actions: {tot}/{len(all_valid_boards)}")

In [ ]:
import dill
import numpy as np
from itertools import product
from collections import defaultdict


class SymmetricQMatrix:
    def __init__(self, file=None, default_value=None):
        self.default_value = 0.0
        # Q-matrix storing canonical state-action pairs
        if default_value is None and file is None:
            self.q_matrix = defaultdict(self._initialize_q_matrix)
        elif default_value is None and file is not None:
            with open(file, 'rb') as f:
                self.q_matrix = dill.load(f)

        elif file is None and default_value is not None:
            self.default_value = default_value
            self.q_matrix = defaultdict(self._initialize_q_matrix)

        # Precompute symmetries and their inverses
        self.transformations = [
            lambda x: x,                         # Identity
            lambda x: np.rot90(x, 1),            # Rotate 90°
            lambda x: np.rot90(x, 2),            # Rotate 180°
            lambda x: np.rot90(x, 3),            # Rotate 270°
            lambda x: np.fliplr(x),              # Horizontal reflection
            lambda x: np.flipud(x),              # Vertical reflection
            lambda x: np.transpose(x),           # Diagonal reflection (TL-BR)
        ]

        self.inverse_transformations = [
            lambda x: x,                         # Identity
            lambda x: np.rot90(x, 3),            # Rotate 90° inverse (rotate 270°)
            lambda x: np.rot90(x, 2),            # Rotate 180° inverse
            lambda x: np.rot90(x, 1),            # Rotate 270° inverse (rotate 90°)
            lambda x: np.fliplr(x),              # Horizontal reflection inverse
            lambda x: np.flipud(x),              # Vertical reflection inverse
            lambda x: np.transpose(x),           # Diagonal reflection (TL-BR) inverse
        ]

        self.original_actions = np.array(range(9)).reshape(3, 3)
        for i, transform in enumerate(self.transformations):
            assert self.inverse_transformations[i](transform(self.original_actions)).flatten().tolist() == self.original_actions.flatten().tolist()

        transformed_actions = [transform(original_actions).flatten().tolist() for transform in transformations]
        for i in range(len(transformed_actions)):
            for j in range(i + 1, len(transformed_actions)):
                assert transformed_actions[i] != transformed_actions[j]

        # Generate all valid Tic-Tac-Toe boards
        self.all_valid_boards = self._generate_all_valid_boards()
        self.all_canonical_boards = set()
        self.get_canonical_boards = {}
        self.get_transform = {}
        self.get_inverse_transform = {}
        self.get_canonical_actions = {}
        self.get_inverse_canonical_actions = {}
        for board in self.all_valid_boards:
            canonical_board, transform_idx = self._get_canonical_representation(board)
            self.all_canonical_boards.add(canonical_board)
            self.get_canonical_boards[board] = canonical_board
            self.get_transform[board] = self.transformations[transform_idx]
            self.get_inverse_transform[board] = self.inverse_transformations[transform_idx]
            self.get_inverse_canonical_actions[board] = self.get_transform[board](self.original_actions).flatten().tolist()
            self.get_canonical_actions[board] = self.get_inverse_transform[board](self.original_actions).flatten().tolist()

        self.all_canonical_boards = list(self.all_canonical_boards)
        self.all_canonical_actions = {}
        for board in self.all_canonical_boards:
            empty_positions = self.get_empty_positions(board)
            self.all_canonical_actions[board] = empty_positions

    def _initialize_q_matrix(self):
        state_dict = defaultdict(lambda: self.default_value)
        return state_dict    

    def _board_to_matrix(self, board):
        """
        Convert a linear board to a 3x3 matrix for easier manipulation
        """
        return np.array(board).reshape(3, 3)

    def _matrix_to_board(self, matrix):
        """
        Convert a 3x3 matrix back to a linear board representation.
        """
        return matrix.flatten().tolist()

    def _generate_all_valid_boards(self):
        symbols = [' ', 'X', 'O']
        all_boards = list(product(symbols, repeat=9))  # Generate all 3^9 combinations
        # print(f"Total number of boards: {len(all_boards)}")
        all_valid_boards = []

        for board in all_boards:
            x_count = board.count('X')
            o_count = board.count('O')
            
            # Valid boards must satisfy these conditions:
            if x_count == o_count or x_count == o_count + 1:
                all_valid_boards.append(board)

        # print(f"Number of valid boards: {len(all_valid_boards)}")    
        return all_valid_boards

    def _generate_symmetries(self, board):
        matrix = self._board_to_matrix(board)
        symmetries = [transform(matrix) for transform in self.transformations]
        return [self._matrix_to_board(sym) for sym in symmetries]

    def _get_canonical_representation(self, board):
        symmetries = self._generate_symmetries(board)
        min_symmetry = min(symmetries)
        return tuple(min_symmetry), symmetries.index(min_symmetry)

    def get_canonical_action(self, board, action):
        return self.get_canonical_actions[tuple(board)][action]

    def get_canonical_board(self, board):
        return self.get_canonical_boards[tuple(board)]

    def get_inverse_canonical_action(self, board, canonical_action):
        return self.get_inverse_canonical_actions[tuple(board)][canonical_action]
    
    def canonicalize(self, board, action):
        canonical_board = self.get_canonical_board(board)
        canonical_action = self.get_canonical_action(board, action)
        return canonical_board, canonical_action

    # Generate all empty positions on the board
    def get_empty_positions(self, board):
        return [i for i, cell in enumerate(board) if cell == ' ']

    def get(self, board=None, action=None):
        """
        Retrieve the Q-value for a state-action pair.
        """
        if board is None and action is None:
            return self.q_matrix
        if action is None and board is not None:
            return self.q_matrix[self.get_canonical_board(board)]
        if action is not None and board is not None:
            canonical_board, canonical_action = self.canonicalize(board, action)
            assert canonical_action in self.all_canonical_actions[canonical_board]
            return self.q_matrix[canonical_board][canonical_action]

    def set(self, board, action, value):
        """
        Set the Q-value for a state-action pair.
        """
        canonical_board, canonical_action = self.canonicalize(board, action)
        assert canonical_action in self.all_canonical_actions[canonical_board]
        self.q_matrix[canonical_board][canonical_action] = value

    def best_actions(self, board, player='X'):
        """
        Choose the best action based on Q-values for the current state.
        """
        actions = self.get_empty_positions(board)

        # Retrieve Q-values for all valid actions
        q_values = {action: self.get(board, action) for action in actions}

        # Choose based on player strategy
        if player == 'X':
            max_q = max(q_values.values())
            best_actions = [action for action, q in q_values.items() if q == max_q]
        else:
            min_q = min(q_values.values())
            best_actions = [action for action, q in q_values.items() if q == min_q]

        return best_actions
    
    def best_action(self, board, player='X'):
        """
        Choose the best action based on Q-values for the current state.
        """
        best_actions = self.best_actions(board, player)
        return np.random.choice(best_actions)

In [ ]:
import random
import time

import numpy as np
from IPython.display import clear_output
from collections import defaultdict

# random.seed(42)  # Set the random seed

def zero_default():
    return 0  # Replace with your desired default value

# Initialize the Tic-Tac-Toe board
def initialize_board():
    return [' ' for _ in range(9)]

# Generate all empty positions on the board
def get_empty_positions(board):
    return [i for i, cell in enumerate(board) if cell == ' ']

# Display the board in a 3x3 format
def display_board(board):
    clear_output(wait=True)
    print("\n")
    print(f" {board[0]}  |  {board[1]}  |  {board[2]} ")
    print("----+-----+----")
    print(f" {board[3]}  |  {board[4]}  |  {board[5]} ")
    print("----+-----+----")
    print(f" {board[6]}  |  {board[7]}  |  {board[8]} ")
    print("\n")

# Display the board in a 3x3 format with Q-values on empty fields
def display_board_with_Q(board, Q, Q_initial_value):
    clear_output(wait=True)
    field = {i: board[i] if board[i] != ' ' else Q.get(board, i) - Q_initial_value for i in range(9)}
    def format_cell(value):
        return f"{value:.1f}" if isinstance(value, (int, float)) else f" {value} "

    print("\n")
    print(f"{format_cell(field[0])} | {format_cell(field[1])} | {format_cell(field[2])}")
    print("----+-----+----")
    print(f"{format_cell(field[3])} | {format_cell(field[4])} | {format_cell(field[5])}")
    print("----+-----+----")
    print(f"{format_cell(field[6])} | {format_cell(field[7])} | {format_cell(field[8])}")
    print("\n")

# Check for a winning condition
def check_winner(board, player):
    win_conditions = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # rows
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # columns
        [0, 4, 8], [2, 4, 6]              # diagonals
    ]
    for condition in win_conditions:
        if all(board[pos] == player for pos in condition):
            return True

    return False

# Check for a draw (no empty spaces)
def check_draw(board):
    return ' ' not in board

# Update Q-values based on the game's outcome, with correct max_future_q
def update_q_values(Q, history, terminal_reward, alpha, gamma):
    diff = 0
    for i in range(len(history) - 1):
        board, action = history[i]
        next_board, _ = history[i + 1]
        
        # Calculate max Q-value for the next state over all possible actions
        next_actions = get_empty_positions(next_board)
        future_qs = [Q.get(next_board, next_action) for next_action in next_actions]
        max_future_q = max(future_qs)
        
        # Update Q-value for current state-action pair
        old_value = Q.get(board, action)
        new_value = (1 - alpha) * old_value + alpha * gamma * max_future_q
        Q.set(board, action, new_value)
        diff += abs(old_value - new_value)

    # Update the last state-action pair with the final reward
    board, action = history[-1]
    old_value = Q.get(board, action)
    new_value = (1 - alpha) * old_value + alpha * terminal_reward
    Q.set(board, action, new_value)
    diff += abs(old_value - new_value)
    return diff

# Choose an action based on Q-values
def choose_action(Q, board, player, epsilon=0.1):
    if random.uniform(0, 1) < epsilon:
        # Exploration: Choose a random move
        empty_positions = get_empty_positions(board)
        return random.choice(empty_positions)
    else:
        # Exploitation: Choose the best known move
        return Q.best_action(board, player)

# Main game loop for two random agents
def play_game(matrices, params, flags):
    if flags is None:
        flags = {}

    training = flags.get('training', True)
    display = flags.get('display', False)
    interactive = flags.get('interactive', False)
    (Q, Visits, Rewards) = matrices
    board = initialize_board()
    current_player = 'X'
    human_player = None
    history = []  # To store state-action pairs
    number_of_actions = 0
    gamma = params['gamma']
    episode = params['episode']
    nr_of_episodes = params['nr_of_episodes']
    epsilon = max(params['epsilon_min'], params['epsilon_start'] / (1 + episode/nr_of_episodes))
    alpha = max(params['alpha_min'], params['alpha_start'] / (1 + episode/nr_of_episodes))

    if interactive:
        while human_player is None:
            user_input = input(f"Choose a player from the set {['X', 'O']}: ")
            human_player = str(user_input)
            if human_player not in ['X', 'O']:
                human_player = None

    while True:
        if display:
            display_board_with_Q(board, Q, params['Q_initial_value'])
            time.sleep(params['waiting_time'])  # Wait a bit before the next move for readability

        empty_positions = get_empty_positions(board)
        if empty_positions:
            if training:
                action = choose_action(Q, board, current_player, epsilon=epsilon)
                history.append((board[:], action))
            else:
                if interactive and current_player == human_player:
                        action = None
                        while action is None:
                            user_input = input(f"Choose a number from the set {empty_positions}: ")
                            action = int(user_input)
                            if action not in empty_positions:
                                action = None
                else:
                    action = choose_action(Q, board, current_player, epsilon=params['eps'][current_player])

            board[action] = current_player
            number_of_actions += 1
            
            # Update Visits
            Visits[tuple(board)][action] += 1

        # Check for winner
        if check_winner(board, current_player):
            terminal_reward = 1.0 if current_player == 'X' else -1.0  # Reward for 'X', penalty for 'O'
            if display:
                display_board(board)
                print(f"Player {current_player} wins!\n")

            if training:
                update_q_values(Q, history, terminal_reward, alpha, gamma)

            # Update Rewards
            Rewards[tuple(board)][action] = terminal_reward

            # Update outcomes
            outcome = current_player
            break
        
        # Check for draw
        if check_draw(board):
            terminal_reward = 0.5
            if display:
                display_board(board)
                print("It's a draw!\n")

            if training:
                update_q_values(Q, history, terminal_reward, alpha, gamma)

            # Update Rewards
            Rewards[tuple(board)][action] = terminal_reward

            # Update outcomes
            outcome = 'D'
            break
        
        # Switch players
        current_player = 'O' if current_player == 'X' else 'X'

    params['outcomes'][outcome] += 1

In [ ]:
# Let the games begin
# Parameters
params = {
    'Q_initial_value' : 1.0, # initial Q value
    'alpha_start' : 0.1,  # initial learning rate
    'alpha_min' : 0.1, # minimum learning rate
    'gamma' : 0.9,  # discount factor
    'epsilon_start' : 0.5,  # initial exploration rate
    'epsilon_min' : 0.25, # minimum exploration rate
    'waiting_time' : 1.0, # waiting in seconds for display
    'episode' : 0, # number of episodes played during training
    'outcomes' : {'X' : 0, 'O' : 0, 'D' : 0}, # dictionary to save the outcomes for games ('X' = X wins, 'O' = O wins, 'D' = draws) 
    'nr_of_episodes' : 500000, # number of episodes for training
    'eps' : {'X' : -1.0, 'O' : -1.0}, # epsilon values for non-training determine how randomized the computer plays
}

# Initialize Q-matrix
Q = SymmetricQMatrix(file='SymmetricQ.pkl')
# Q = SymmetricQMatrix(default_value=params['Q_initial_value'])
Visits = defaultdict(lambda: defaultdict(zero_default))
Rewards = defaultdict(lambda: defaultdict(zero_default))

matrices = (Q, Visits, Rewards)

nr_of_episodes = 50000
params['nr_of_episodes'] = nr_of_episodes
params['episode'] = 0

flags = {
    'training': True,
    'display': False,
    'interactive': False
}
for _ in range(nr_of_episodes):
    play_game(matrices, params, flags=flags)
    params['episode'] += 1

print("Outcomes:")
print(f"X wins: {params['outcomes']['X']/nr_of_episodes}, O wins: {params['outcomes']['O']/nr_of_episodes}, draws: {params['outcomes']['D']/nr_of_episodes}")

In [ ]:
nr_of_episodes = 5000
params['nr_of_episodes'] = nr_of_episodes
params['eps'] = {'X' : 0.1, 'O' : -0.1}
params['outcomes'] = {'X' : 0, 'O' : 0, 'D' : 0}
flags = {
    'training': False,
    'display': False,
    'interactive': False
}
for _ in range(nr_of_episodes):
    play_game(matrices, params, flags=flags)

print("Outcomes with players choosing action based on Q-values:")
print(f"X wins: {params['outcomes']['X']/nr_of_episodes}, O wins: {params['outcomes']['O']/nr_of_episodes}, draws: {params['outcomes']['D']/nr_of_episodes}")

Outcomes with players choosing action based on Q-values:
X wins: 0.138, O wins: 0.6906, draws: 0.1714

In [ ]:
# import dill

# with open('SymmetricQ.pkl', 'wb') as f:
#     dill.dump(Q.get(), f)

In [ ]:
flags = {
    'training': False,
    'display': True,
    'interactive': False
}
play_game(matrices, params, flags=flags)

In [ ]:
flags = {
    'training': False,
    'display': True,
    'interactive': True
}
# play_game(matrices, params, flags=flags)

In [ ]:
q_matrix = Q.get()
for canonical_board in Q.all_canonical_boards:
    setA = set(q_matrix[canonical_board].keys()) - set(Q.all_canonical_actions[canonical_board])
    setB = set(Q.all_canonical_actions[canonical_board]) - set(q_matrix[canonical_board].keys())
    if len(setA) > 0:
        print(canonical_board)

In [ ]:
import matplotlib.pyplot as plt

q_matrix = Q.get()
print(f"Total unique states encountered: {len(q_matrix.keys())}")

# Extract all Q-values from the nested dictionary
all_q_values = [q for actions in q_matrix.values() for q in actions.values()]
print(f"Total number of elements in Q: {len(all_q_values)}")

mean_q = np.mean(all_q_values)
median_q = np.median(all_q_values)
std_q = np.std(all_q_values)
min_q = np.min(all_q_values)
max_q = np.max(all_q_values)

print("Q-value Statistics:")
print(f"Mean: {mean_q}")
print(f"Median: {median_q}")
print(f"Standard Deviation: {std_q}")
print(f"Minimum: {min_q}")
print(f"Maximum: {max_q}")

plt.figure(figsize=(10, 6))
plt.hist(all_q_values, bins=20, edgecolor='black', alpha=0.7)
plt.title("Histogram of Q-values")
plt.xlabel("Q-value")
plt.ylabel("Frequency")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# with open('Q_optimal.pkl', 'wb') as f:
#     dill.dump(Q, f)

Number of canonical boards: 1520
Number of canonical state-action pairs: 4808

In [ ]:
all_v = [v for states in Visits.values() for v in states.values()]

print("Statistics of visited state-action pairs:")
print(f"Number of state-action pairs visited: {len(all_v)}")

mean_v = np.mean(all_v)
median_v = np.median(all_v)
std_v = np.std(all_v)
min_v = np.min(all_v)
max_v = np.max(all_v)

print(f"Mean: {mean_v}")
print(f"Median: {median_v}")
print(f"Standard Deviation: {std_v}")
print(f"Minimum: {min_v}")
print(f"Maximum: {max_v}")

plt.figure(figsize=(10, 6))
plt.hist(all_v, bins=20, edgecolor='black', alpha=0.7)
plt.title("Histogram of visited states")
plt.ylabel("Visits")
plt.xlabel("States")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()